---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [3]:
pd.set_option('display.max_rows', 500)

In [4]:
pd.set_option('display.max_colwidth', -1)

In [5]:
df2 = pd.DataFrame(df, columns = ["text"])
df2.head()

,text
0,03/25/93 Total time of visit (in minutes):\n
1,6/18/85 Primary Care Doctor:\n
2,sshe plans to move as of 7/8/71 In-Home Services: None\n
3,7 on 9/27/75 Audit C Score Current:\n
4,2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7\n


In [6]:
df2["text2"] = df2.text.str.lower()
df2["text2"] = df2.text2.str.split().str.join(" ")

In [ ]:
df2.text2

In [ ]:
df2['text2'].str.extract(
                            r'(?:^|\D)([012]*[0-9]|3[01]|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[-/,. ]{1,2}'
                            r'(\d{1,2}|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[-/,. ]{1,2}'
                            r'([56789][0-9]|19[0-9][0-9]|20[0-9][0-9])(?:$|\D)')

In [107]:
# full date
df2[['month','day','year']] = (df2['text2'].str.extract(
                            r'(?:^|\D)([012]*[0-9]|3[01]|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[-/,. ]{1,2}'
                            r'(\d{1,2}|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[-/,. ]{1,2}'
                            r'([56789][0-9]|19[0-9][0-9]|20[0-9][0-9])(?:$|\D)').rename(columns = {0:'month', 1:'day', 2:'year'}))
# only month and year 
month_year_fill = df2.loc[((df2.year.isnull()) &(df2.month.isnull())),'text2'].str.extract(
                            r'(?:^|\D)([0-9]|10|11|12|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[-/,. ]{1,2}'
                            r'(19[0-9][0-9]|20[0-9][0-9])(?:$|\D)').rename(columns = {0:'month', 1:'year'})
month_year_fill.dropna(inplace = True)
df2.loc[month_year_fill.index, "month"] =  month_year_fill.month
df2.loc[month_year_fill.index, "year"]  = month_year_fill.year
df2.loc[month_year_fill.index, "day"]   = 1
# only year
year_fill = df2.loc[df2.year.isnull(),'text2'].str.extract(r'(19[0-9][0-9]|20[0-9][0-9])').rename(columns = {0:'year'})
df2.loc[year_fill.index, "month"] =  'jan'
df2.loc[year_fill.index, "year"]  = year_fill
df2.loc[year_fill.index, "day"]   = 1
# flip days and months for wrong ordered ones
flip = df2.loc[df2.day.str.len() == 3].copy()
df2.loc[flip.index, "month"] = flip.day
df2.loc[flip.index, "day" ]  = flip.month
# convert months to integers
month_map = {"jan" : "1", "feb" : "2", "mar" : "3", "apr" : "4", "may" : "5", "jun" : "6",
             "jul" : "7", "aug" : "8", "sep" : "9", "oct" : "10", "nov" : "11", "dec" : "12"}
for month, num in month_map.items():
    df2.month = df2.month.str.replace(month, num)
# clean numbers
df2.month = df2.month.astype("int").astype("str")
df2.day = df2.day.astype("int").astype("str")
df2.loc[df2.year.str.len() == 2, "year"]   = "19" + df2.loc[df2.year.str.len() == 2, "year"]
df2.loc[df2.month.str.len() == 1, "month"] = "0"  + df2.loc[df2.month.str.len() == 1, "month"]
df2.loc[df2.day.str.len() == 1, "day"]     = "0"  + df2.loc[df2.day.str.len() == 1, "day"]

# create final dates and return
df2["dates"] = df2.year + df2.month + df2.day
df2.dates = df2.dates.astype("int")
pd.Series(df2.dates.sort_values(ascending = False).index.values)

In [ ]:
pd.Series(df2.dates.sort_values(ascending = False).index.values)

In [102]:
df2.month.unique()

array(['3', '6', '7', '9', '2', '5', '10', '4', '8', '1', '11', '12'], dtype=object)

In [103]:
df2.day.unique()

array(['25', '18', '8', '27', '6', '24', '7', '10', '11', '9', '1', '26',
       '12', '13', '21', '3', '22', '2', '31', '20', '15', '14', '29',
       '16', '17', '28', '5', '4', '19', '30', '23'], dtype=object)

In [106]:
df2.year.unique()

array(['1993', '1985', '1971', '1975', '1996', '1979', '1978', '1989',
       '1986', '1998', '1972', '1990', '2011', '1982', '1976', '1977',
       '1984', '1987', '1991', '2012', '1983', '1973', '1988', '1981',
       '1994', '2002', '1992', '1995', '2010', '1974', '1999', '1980',
       '2000', '1997', '2004', '2006', '2001', '2007', '2016', '2003',
       '2015', '2008', '2013', '2005', '2014', '2009'], dtype=object)

In [3]:
def date_sorter():
    import warnings
    warnings.filterwarnings('ignore')

    import pandas as pd

    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)
    
    # Your code here
    df2 = pd.DataFrame(df, columns = ["text"])
    
    df2["text2"] = df2.text.str.lower()
    df2["text2"] = df2.text2.str.split().str.join(" ")
    # full date
    df2[['month','day','year']] = (df2['text2'].str.extract(
                                r'(?:^|\D)([012]*[0-9]|3[01]|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[-/,. ]{1,2}'
                                r'(\d{1,2}|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[-/,. ]{1,2}'
                                r'([56789][0-9]|19[0-9][0-9]|20[0-9][0-9])(?:$|\D)').rename(columns = {0:'month', 1:'day', 2:'year'}))
    # only month and year 
    month_year_fill = df2.loc[((df2.year.isnull()) &(df2.month.isnull())),'text2'].str.extract(
                                r'(?:^|\D)([0-9]|10|11|12|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[-/,. ]{1,2}'
                                r'(19[0-9][0-9]|20[0-9][0-9])(?:$|\D)').rename(columns = {0:'month', 1:'year'})
    month_year_fill.dropna(inplace = True)
    df2.loc[month_year_fill.index, "month"] =  month_year_fill.month
    df2.loc[month_year_fill.index, "year"]  = month_year_fill.year
    df2.loc[month_year_fill.index, "day"]   = 1
    # only year
    year_fill = df2.loc[df2.year.isnull(),'text2'].str.extract(r'(19[0-9][0-9]|20[0-9][0-9])').rename(columns = {0:'year'})
    df2.loc[year_fill.index, "month"] =  'jan'
    df2.loc[year_fill.index, "year"]  = year_fill
    df2.loc[year_fill.index, "day"]   = 1
    # flip days and months for wrong ordered ones
    flip = df2.loc[df2.day.str.len() == 3].copy()
    df2.loc[flip.index, "month"] = flip.day
    df2.loc[flip.index, "day" ]  = flip.month
    # convert months to integers
    month_map = {"jan" : "1", "feb" : "2", "mar" : "3", "apr" : "4", "may" : "5", "jun" : "6",
                 "jul" : "7", "aug" : "8", "sep" : "9", "oct" : "10", "nov" : "11", "dec" : "12"}
    for month, num in month_map.items():
        df2.month = df2.month.str.replace(month, num)
    # clean numbers
    df2.month = df2.month.astype("int").astype("str")
    df2.day = df2.day.astype("int").astype("str")
    df2.loc[df2.year.str.len() == 2, "year"]   = "19" + df2.loc[df2.year.str.len() == 2, "year"]
    df2.loc[df2.month.str.len() == 1, "month"] = "0"  + df2.loc[df2.month.str.len() == 1, "month"]
    df2.loc[df2.day.str.len() == 1, "day"]     = "0"  + df2.loc[df2.day.str.len() == 1, "day"]

    # create final dates and return
    df2["dates"] = df2.year + df2.month + df2.day
    df2.dates = df2.dates.astype("int")

    return  pd.Series(df2.dates.sort_values(ascending = False).index.values) # Your answer here